<a href="https://colab.research.google.com/github/rlagusdn04/XRPL/blob/main/notebooks/colab-github-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Google Colab with GitHub




[Google Colaboratory](http://colab.research.google.com) is designed to integrate cleanly with GitHub, allowing both loading notebooks from github and saving notebooks to github.

## Loading Public Notebooks Directly from GitHub

Colab can load public github notebooks directly, with no required authorization step.

For example, consider the notebook at this address: https://github.com/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb.

The direct colab link to this notebook is: https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb.

To generate such links in one click, you can use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) Chrome extension.

## Browsing GitHub Repositories from Colab

Colab also supports special URLs that link directly to a GitHub browser for any user/organization, repository, or branch. For example:

- http://colab.research.google.com/github will give you a general github browser, where you can search for any github organization or username.
- http://colab.research.google.com/github/googlecolab/ will open the repository browser for the ``googlecolab`` organization. Replace ``googlecolab`` with any other github org or user to see their repositories.
- http://colab.research.google.com/github/googlecolab/colabtools/ will let you browse the main branch of the ``colabtools`` repository within the ``googlecolab`` organization. Substitute any user/org and repository to see its contents.
- http://colab.research.google.com/github/googlecolab/colabtools/blob/main will let you browse ``main`` branch of the ``colabtools`` repository within the ``googlecolab`` organization. (don't forget the ``blob`` here!) You can specify any valid branch for any valid repository.

## Loading Private Notebooks

Loading a notebook from a private GitHub repository is possible, but requires an additional step to allow Colab to access your files.
Do the following:

1. Navigate to http://colab.research.google.com/github.
2. Click the "Include Private Repos" checkbox.
3. In the popup window, sign-in to your Github account and authorize Colab to read the private files.
4. Your private repositories and notebooks will now be available via the github navigation pane.

## Saving Notebooks To GitHub or Drive

Any time you open a GitHub hosted notebook in Colab, it opens a new editable view of the notebook. You can run and modify the notebook without worrying about overwriting the source.

If you would like to save your changes from within Colab, you can use the File menu to save the modified notebook either to Google Drive or back to GitHub. Choose **File→Save a copy in Drive** or **File→Save a copy to GitHub** and follow the resulting prompts. To save a Colab notebook to GitHub requires giving Colab permission to push the commit to your repository.

## Open In Colab Badge

Anybody can open a copy of any github-hosted notebook within Colab. To make it easier to give people access to live views of GitHub-hosted notebooks,
colab provides a [shields.io](http://shields.io/)-style badge, which appears as follows:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)

The markdown for the above badge is the following:

```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
```

The HTML equivalent is:

```HTML
<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
```

Remember to replace the notebook URL in this template with the notebook you want to link to.

In [ ]:
#!git clone https://github.com/rlagusdn04/XRPL

import os
os.chdir("/content")  # 저장소 경로로 이동
!pwd

from google.colab import drive
drive.mount('/content/drive')



/content
Mounted at /content/drive


In [6]:
# 1. nest_asyncio와 xrpl-py 설치
!pip install nest_asyncio xrpl-py

# 2. 설치된 패키지 정보 확인 (xrpl-py)
!pip show xrpl-py

# 3. nest_asyncio 적용 (Colab에서 이미 실행중인 event loop에 문제 없도록 패치)
import nest_asyncio
nest_asyncio.apply()

# 4. 필요한 모듈 임포트
from xrpl.clients import JsonRpcClient
from xrpl.wallet import generate_faucet_wallet
from xrpl.account import get_balance

# 5. XRPL 테스트넷 엔드포인트 설정 및 연결 확인
testnet_url = "https://s.altnet.rippletest.net:51234/"
client = JsonRpcClient(testnet_url)

server_info = client.request({"method": "server_info"})
print("서버 정보:", server_info.result)

# 6. 새로운 테스트넷 지갑 생성 (Faucet을 이용)
wallet = generate_faucet_wallet(client)
print(f"지갑 주소: {wallet.classic_address}")
print(f"비밀키: {wallet.seed}")

# 7. 생성된 지갑의 XRP 잔액 조회
balance = get_balance(wallet.classic_address, client)
print(f"잔액: {balance} XRP")


Name: xrpl-py
Version: 4.1.0
Summary: A complete Python library for interacting with the XRP ledger
Home-page: https://github.com/XRPLF/xrpl-py
Author: Mayukha Vadari
Author-email: mvadari@ripple.com
License: ISC
Location: /usr/local/lib/python3.11/dist-packages
Requires: base58, Deprecated, ECPy, httpx, pycryptodome, types-Deprecated, typing-extensions, websockets
Required-by: 


<frozen zipimport>:75: RuntimeWarning: coroutine 'JsonRpcBase._request_impl' was never awaited


AttributeError: 'dict' object has no attribute 'to_dict'